In [1]:
import pandas as pd
import numpy as np

# 定义 ID3 决策树算法
def ID3(data, original_data, features, target_attribute_name, parent_node_class=None):
    # 如果所有目标值相同，则返回这个值
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    # 如果数据集为空，返回父节点的目标值
    elif len(data) == 0:
        return parent_node_class
    
    # 如果特征集为空，则返回数据集中出现最频繁的目标值
    elif len(features) == 0:
        return np.unique(original_data[target_attribute_name])[np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])]
    
    # 构建决策树
    else:
        # 设置父节点的目标值
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
        
        # 选择最佳分割特征
        best_feature = get_best_feature(data, features, target_attribute_name)
        
        # 构建树
        tree = {best_feature: {}}
        
        # 移除已选择的特征
        features = [i for i in features if i != best_feature]
        
        # 递归构建树
        for value in np.unique(data[best_feature]):
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data, original_data, features, target_attribute_name, parent_node_class)
            tree[best_feature][value] = subtree
            
        return tree

# 获取最佳分割特征
def get_best_feature(data, features, target_attribute_name):
    # 计算信息增益
    info_gain = []
    for feature in features:
        info_gain.append(calculate_information_gain(data, feature, target_attribute_name))
    return features[np.argmax(info_gain)]

# 计算信息增益
def calculate_information_gain(data, split_attribute_name, target_attribute_name):
    # 计算数据集的熵
    total_entropy = calculate_entropy(data[target_attribute_name])
    
    # 计算分割属性的熵和信息增益
    values, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = 0
    for i in range(len(values)):
        subset = data.where(data[split_attribute_name] == values[i]).dropna()
        weighted_entropy += (counts[i] / np.sum(counts)) * calculate_entropy(subset[target_attribute_name])
    
    return total_entropy - weighted_entropy

# 计算熵
def calculate_entropy(target):
    elements, counts = np.unique(target, return_counts=True)
    entropy = -np.sum([(counts[i] / np.sum(counts)) * np.log2(counts[i] / np.sum(counts)) for i in range(len(elements))])
    return entropy

# 对测试集进行预测
def predict(tree, test_data):
    predictions = []
    for index, row in test_data.iterrows():
        prediction = classify(tree, row)
        predictions.append(prediction)
    return predictions

# 对单个样本进行分类
def classify(node, sample):
    if isinstance(node, dict):
        key = list(node.keys())[0]
        value = sample[key]
        if value in node[key]:
            return classify(node[key][value], sample)
        else:
            return "Unknown"
    else:
        return node

# 读取数据集
train_data = pd.read_csv('Watermelon-train1.csv',encoding="gb2312")  # 读取训练集
test_data = pd.read_csv('Watermelon-test1.csv',encoding="gb2312")  # 读取测试集

# 获取特征和目标列
features = train_data.columns[:-1]
target_attribute_name = train_data.columns[-1]
print(target_attribute_name)

# 构建决策树
tree = ID3(train_data, train_data, features, target_attribute_name)
print(tree)

# 对测试集进行预测
predictions = predict(tree, test_data)

# 计算分类精度
accuracy = np.mean(predictions == test_data[target_attribute_name]) * 100
print("分类精度：", accuracy)


好瓜
{'编号': {1: '是', 2: '是', 3: '是', 4: '是', 5: '是', 6: '是', 7: '是', 8: '是', 9: '否', 10: '否', 11: '否', 12: '否', 13: '否', 14: '否', 15: '否', 16: '否'}}
分类精度： 70.0


In [6]:
import pandas as pd
import numpy as np

#计算信息熵
def cal_information_entropy(data):
    data_label = data.iloc[:,-1]
    label_class =data_label.value_counts() #总共有多少类
    Ent = 0
    for k in label_class.keys():
        p_k = label_class[k]/len(data_label)
        Ent += -p_k*np.log2(p_k)
    return Ent

#计算给定数据属性a的信息增益
def cal_information_gain(data, a):
    Ent = cal_information_entropy(data)
    feature_class = data[a].value_counts() #特征有多少种可能
    gain = 0
    for v in feature_class.keys():
        weight = feature_class[v]/data.shape[0]
        Ent_v = cal_information_entropy(data.loc[data[a] == v])
        gain += weight*Ent_v
    return Ent - gain

def cal_gain_ratio(data , a):
    #先计算固有值intrinsic_value
    IV_a = 0
    feature_class = data[a].value_counts()  # 特征有多少种可能
    for v in feature_class.keys():
        weight = feature_class[v]/data.shape[0]
        IV_a += -weight*np.log2(weight)
    gain_ration = cal_information_gain(data,a)/IV_a
    return gain_ration


In [8]:

#获取标签最多的那一类
def get_most_label(data):
    data_label = data.iloc[:,-1]
    label_sort = data_label.value_counts(sort=True)
    return label_sort.keys()[0]

#挑选最优特征，即在信息增益大于平均水平的特征中选取增益率最高的特征
def get_best_feature(data):
    features = data.columns[:-1]
    res = {}
    for a in features:
        temp = cal_information_gain(data, a)
        print("1",temp)
        gain_ration = cal_gain_ratio(data,a)
        print("2",gain_ration)
        res[a] = (temp,gain_ration)
    res = sorted(res.items(),key=lambda x:x[1][0],reverse=True) #按信息增益排名
    res_avg = sum([x[1][0] for x in res])/len(res) #信息增益平均水平
    good_res = [x for x in res if x[1][0] >= res_avg] #选取信息增益高于平均水平的特征
    result =sorted(good_res,key=lambda x:x[1][1],reverse=True) #将信息增益高的特征按照增益率进行排名
    return result[0][0] #返回高信息增益中增益率最大的特征

##将数据转化为（属性值：数据）的元组形式返回，并删除之前的特征列
def drop_exist_feature(data, best_feature):
    attr = pd.unique(data[best_feature])
    new_data = [(nd, data[data[best_feature] == nd]) for nd in attr]
    new_data = [(n[0], n[1].drop([best_feature], axis=1)) for n in new_data]
    return new_data

#创建决策树
def create_tree(data):
    data_label = data.iloc[:,-1]
    if len(data_label.value_counts()) == 1: #只有一类
        return data_label.values[0]
    if all(len(data[i].value_counts()) == 1 for i in data.iloc[:,:-1].columns): #所有数据的特征值一样，选样本最多的类作为分类结果
        return get_most_label(data)
    best_feature = get_best_feature(data) #根据信息增益得到的最优划分特征
    Tree = {best_feature:{}} #用字典形式存储决策树
    exist_vals = pd.unique(data[best_feature])  # 当前数据下最佳特征的取值
    if len(exist_vals) != len(column_count[best_feature]):  # 如果特征的取值相比于原来的少了
        no_exist_attr = set(column_count[best_feature]) - set(exist_vals)  # 少的那些特征
        for no_feat in no_exist_attr:
            Tree[best_feature][no_feat] = get_most_label(data)  # 缺失的特征分类为当前类别最多的
    for item in drop_exist_feature(data,best_feature): #根据特征值的不同递归创建决策树
        Tree[best_feature][item[0]] = create_tree(item[1])
    return Tree

def predict(Tree , test_data):
    first_feature = list(Tree.keys())[0]
    second_dict = Tree[first_feature]
    input_first = test_data.get(first_feature)
    input_value = second_dict[input_first]
    if isinstance(input_value , dict): #判断分支还是不是字典
        class_label = predict(input_value, test_data)
    else:
        class_label = input_value
    return class_label

if __name__ == '__main__':
    #读取数据
    data = pd.read_csv('Watermelon-train2.csv',encoding="gb2312")
    # 统计每个特征的取值情况作为全局变量
    column_count = dict([(ds, list(pd.unique(data[ds]))) for ds in data.iloc[:, :-1].columns])

    #创建决策树
    dicision_Tree = create_tree(data)
    print(dicision_Tree)
    #测试数据
    test_data_1 = {'色泽':'青绿','根蒂':'蜷缩','敲声':'浊响','纹理':'稍糊','脐部':'凹陷','触感':'硬滑'}
    test_data_2 = pd.read_csv('Watermelon-test2.csv',encoding="gb2312") 
    result = predict(dicision_Tree,test_data_2)
    print('分类结果为'+'好瓜'if result == 1 else '坏瓜')

1 0.9975025463691153
2 0.24403953873351492
1 0.10812516526536531
2 0.06843956584615814
1 0.14267495956679288
2 0.1017593980537369
1 0.14078143361499584
2 0.10562670944314426
1 0.3805918973682686
2 0.2630853587192754
1 0.9975025463691153
2 0.24403953873351492
{'编号': {1: '是', 2: '是', 3: '是', 4: '是', 5: '是', 6: '是', 7: '是', 8: '是', 9: '否', 10: '否', 11: '否', 12: '否', 13: '否', 14: '否', 15: '否', 16: '否', 17: '否'}}


TypeError: unhashable type: 'Series'